In [ ]:
'''Competitions'''
import mysql.connector
import requests
import json

# 📌 API Request to SportRadar
url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=W9Trt4ohZK8Gj6fUxVoPn5sTMsmCl48Rl0RetWiJ"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

# ✅ Check if API request is successful
if response.status_code == 200:
    data = response.json()
    print("✅ Data fetched successfully!")
    competitions = data.get("competitions", [])
    print(f"📌 Total competitions found: {len(competitions)}")
else:
    print(f"❌ API request failed! Status Code: {response.status_code}")
    competitions = []

# ✅ Connect to MySQL Database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Priya09@2001",
    database="sportsradar",
    autocommit=True
)
cursor = conn.cursor()
cursor.execute("SET SESSION innodb_lock_wait_timeout = 5;")

# ✅ Insert Categories & Competitions into MySQL
for comp in competitions:
    category_id = comp["category"]["id"].split(":")[-1]  # Extract numeric ID
    category_name = comp["category"]["name"]
    comp_id = comp["id"]
    comp_name = comp["name"]

    # ✅ Handle missing fields (Prevent NULL values)
    comp_type = comp.get("type", "unknown")  # Default "unknown" if missing
    comp_gender = comp.get("gender", "unknown")  # Default "unknown" if missing
    comp_level = comp.get("level", "unknown")  # Default "unknown" if missing

    # 🔹 Insert category into `categories` table
    cursor.execute("""
        INSERT IGNORE INTO categories (id, name)
        VALUES (%s, %s);
    """, (category_id, category_name))

    # 🔹 Insert competition into `competitions` table
    cursor.execute("""
        INSERT IGNORE INTO competitions (id, name, type, gender, category_id, level)
        VALUES (%s, %s, %s, %s, %s, %s);
    """, (comp_id, comp_name, comp_type, comp_gender, category_id, comp_level))

# ✅ Close the connection
cursor.close()
conn.close()

print("✅ Data inserted successfully into MySQL!")


✅ Data fetched successfully!
📌 Total competitions found: 5928
✅ Data inserted successfully into MySQL!


In [7]:
import mysql.connector

# ✅ Connect to MySQL Database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Priya09@2001",
    database="sportsradar",
    autocommit=True  # Ensures changes are committed immediately
)
cursor = conn.cursor()

# ✅ Remove all rows (without deleting the table)
cursor.execute("DELETE FROM competitions;")
cursor.execute("DELETE FROM categories;")

# ✅ Commit changes
conn.commit()

print("✅ All values removed from tables!")

# ✅ Close connection
cursor.close()
conn.close()


✅ All values removed from tables!


In [ ]:
'''complexes'''
import mysql.connector
import requests
import json

# 📌 API Request to SportRadar
url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=W9Trt4ohZK8Gj6fUxVoPn5sTMsmCl48Rl0RetWiJ"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

# ✅ Check if API request is successful
if response.status_code == 200:
    data = response.json()
    complexes = data.get("complexes", [])  # Extract complexes list
else:
    print(f"❌ API request failed! Status Code: {response.status_code}")
    exit()

# ✅ Connect to MySQL Database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Priya09@2001",  # Change this to your actual password
    database="sportsradar"
)
cursor = conn.cursor()

# ✅ Create Tables if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS complexes (
        complex_id VARCHAR(50) PRIMARY KEY,
        complex_name VARCHAR(100) NOT NULL
    );
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS venues (
        venue_id VARCHAR(50) PRIMARY KEY,
        venue_name VARCHAR(100) NOT NULL,
        city_name VARCHAR(100) NOT NULL,
        country_name VARCHAR(100) NOT NULL,
        country_code CHAR(3) NOT NULL,
        timezone VARCHAR(100) NOT NULL,
        complex_id VARCHAR(50),
        FOREIGN KEY (complex_id) REFERENCES complexes(complex_id)
    );
""")

# ✅ Insert Data into MySQL
for complex_data in complexes:
    if "id" not in complex_data or "name" not in complex_data or "venues" not in complex_data:
        continue  # 🔹 Skip incomplete complexes

    complex_id = complex_data["id"]
    complex_name = complex_data["name"]

    # 🔹 Insert into `complexes` table
    cursor.execute("""
        INSERT IGNORE INTO complexes (complex_id, complex_name)
        VALUES (%s, %s);
    """, (complex_id, complex_name))

    # 🔹 Insert into `venues` table
    for venue in complex_data["venues"]:
        if not all(k in venue for k in ["id", "name", "city_name", "country_name", "country_code", "timezone"]):
            continue  # 🔹 Skip incomplete venues

        venue_id = venue["id"]
        venue_name = venue["name"]
        city_name = venue["city_name"]
        country_name = venue["country_name"]
        country_code = venue["country_code"]
        timezone = venue["timezone"]

        cursor.execute("""
            INSERT IGNORE INTO venues (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id))

# ✅ Commit & Close
conn.commit()
cursor.close()
conn.close()

print("✅ Data inserted successfully into MySQL!")


✅ Data inserted successfully into MySQL!


In [ ]:
'''double_competitors_rankings'''
import mysql.connector
import requests
import json

# ✅ API Request
url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=W9Trt4ohZK8Gj6fUxVoPn5sTMsmCl48Rl0RetWiJ"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

# ✅ Check API response
if response.status_code == 200:
    data = response.json()
    print("✅ Data fetched successfully!")

    # Extract rankings data
    rankings = []
    if "rankings" in data:
        for ranking in data["rankings"]:
            if "competitor_rankings" in ranking:
                rankings.extend(ranking["competitor_rankings"])  # Flatten list

    print(f"📌 Total rankings found: {len(rankings)}")
else:
    print(f"❌ API request failed! Status Code: {response.status_code}")
    exit()

# ✅ Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Priya09@2001",
    database="sportsradar",
    autocommit=True
)
cursor = conn.cursor()

# ✅ Insert Data into MySQL
for record in rankings:
    rank = record["rank"]
    movement = record["movement"]
    points = record["points"]
    competitions_played = record["competitions_played"]
    
    competitor = record["competitor"]
    competitor_id = competitor["id"]
    name = competitor["name"]
    country = competitor.get("country", "Unknown")  # ✅ Handle missing country
    country_code = competitor.get("country_code", "N/A")  # ✅ Handle missing country_code
    abbreviation = competitor.get("abbreviation", "N/A")  # ✅ Handle missing abbreviation

    # 🔹 Insert into `competitors` table
    cursor.execute("""
        INSERT IGNORE INTO competitors (competitor_id, name, country, country_code, abbreviation)
        VALUES (%s, %s, %s, %s, %s);
    """, (competitor_id, name, country, country_code, abbreviation))

    # 🔹 Insert into `competitor_rankings` table
    cursor.execute("""
        INSERT INTO competitor_rankings (`rank`, movement, points, competitions_played, competitor_id)
        VALUES (%s, %s, %s, %s, %s);
    """, (rank, movement, points, competitions_played, competitor_id))

# ✅ Close MySQL Connection
cursor.close()
conn.close()

print("✅ Data inserted successfully into MySQL!")


✅ Data fetched successfully!
📌 Total rankings found: 1000
✅ Data inserted successfully into MySQL!


In [23]:
import mysql.connector

# ✅ Connect to MySQL Database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Priya09@2001",
    database="sportsradar",
    autocommit=True  # Ensures changes are committed immediately
)
cursor = conn.cursor()

# ✅ Remove all rows (without deleting the table)
cursor.execute("DELETE FROM competitions;;")


# ✅ Commit changes
conn.commit()

print("✅ All values removed from tables!")

# ✅ Close connection
cursor.close()
conn.close()


✅ All values removed from tables!


In [9]:
import mysql.connector
import pandas as pd

# ✅ Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Priya09@2001",  # Replace with your actual password
    database="sportsradar"
)

# ✅ Define Queries for Each Table
queries = {
    "competitors": "SELECT * FROM competitors;",
    "competitor_rankings": "SELECT * FROM competitor_rankings;",
    "competitions": "SELECT * FROM competitions;",
    "categories": "SELECT * FROM categories;",
    "venues": "SELECT * FROM venues;",
    "complexes": "SELECT * FROM complexes;"
}

# ✅ Load Data into Pandas DataFrames
dataframes = {table: pd.read_sql(query, conn) for table, query in queries.items()}

# ✅ Close MySQL Connection
conn.close()

# ✅ Display the First 5 Rows of Each Table
for table_name, df in dataframes.items():
    print(f"\n🔹 {table_name.upper()} (First 5 Rows):\n", df.head())



🔹 COMPETITORS (First 5 Rows):
            competitor_id               name  country country_code  \
0   sr:competitor:100069    Sachko, Vitaliy  Ukraine          UKR   
1   sr:competitor:100341  Bolkvadze, Mariam  Georgia          GEO   
2  sr:competitor:1017327      Samson, Laura  Czechia          CZE   
3  sr:competitor:1020177       Huang, Yujia    China          CHN   
4  sr:competitor:1020797      Trhac, Patrik      USA          USA   

  abbreviation  type  
0          SAC  None  
1          BOL  None  
2          SAM  None  
3          HUA  None  
4          TRH  None  

🔹 COMPETITOR_RANKINGS (First 5 Rows):
    rank_id  rank  movement  points  competitions_played         competitor_id
0        1     1         0    7620                   23   sr:competitor:49363
1        2     1         0    7620                   23   sr:competitor:51836
2        3     3         0    7355                   28  sr:competitor:637970
3        4     4         0    7205                   26   sr:co

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21240\2152900013.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes = {table: pd.read_sql(query, conn) for table, query in queries.items()}


In [ ]:
'''Model Building -- Random Forest for Predicting Players Rankings'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ✅ Load DataFrames
competitor_rankings = dataframes["competitor_rankings"]
competitors = dataframes["competitors"]

# ✅ Merge competitor_rankings with competitors on competitor_id
df = competitor_rankings.merge(competitors, on="competitor_id", how="left")

# ✅ Drop unnecessary columns
df.drop(columns=["type", "name", "abbreviation"], inplace=True)

# ✅ Handle missing values in country & country_code
if df[['country', 'country_code']].isnull().sum().sum() > 0:
    print("Missing values found in country columns. Dropping them...")
    df.drop(columns=['country', 'country_code'], inplace=True)

# ✅ Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
if len(categorical_cols) > 0:
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    encoded_cols = encoder.fit_transform(df[categorical_cols])
    encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(categorical_cols))
    df = pd.concat([df.drop(columns=categorical_cols), encoded_df], axis=1)

# ✅ Define Features (X) and Target Variable (y)
X = df.drop(columns=['rank'])
y = df['rank']

# ✅ Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ✅ Train Random Forest Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ✅ Predictions
y_pred = model.predict(X_test)

# ✅ Evaluate Performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\U0001F4CA Model Performance:\n"
      f"Mean Absolute Error (MAE): {mae:.2f}\n"
      f"Mean Squared Error (MSE): {mse:.2f}\n"
      f"R² Score: {r2:.4f}")

# ✅ Feature Importance
feature_importance = model.feature_importances_
features = np.array(X.columns)

sorted_idx = np.argsort(feature_importance)[::-1]
features = features[sorted_idx]
importance = feature_importance[sorted_idx]

plt.figure(figsize=(12, 6))
plt.barh(features[:10], importance[:10], color='royalblue')
plt.xlabel("Feature Importance Score")
plt.ylabel("Features")
plt.title("Top 10 Important Features in RandomForest Model")
plt.gca().invert_yaxis()
plt.show()

# ✅ Feature Correlation Heatmap
plt.figure(figsize=(10,6))
sns.heatmap(df.corr(), cmap="coolwarm", annot=False)
plt.title("Feature Correlation Heatmap")
plt.show()

# ✅ Save the trained model
with open("sports_radar_model.pkl", "wb") as file:
    pickle.dump(model, file)

In [2]:
import pickle
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

# Simulate some example data (Replace with actual dataset)
np.random.seed(42)
data = {
    'points': np.random.randint(100, 1000, 100),
    'competitions_played': np.random.randint(1, 50, 100),
    'movement': np.random.randint(-10, 10, 100),
    'rank': np.random.randint(1, 100, 100)
}
df = pd.DataFrame(data)

# Define features and target
X = df[['points', 'competitions_played', 'movement']]
y = df['rank']

# Train the model
model = RandomForestRegressor()
model.fit(X, y)

# Save the model properly
with open("sports_radar_model.pkl", "wb") as file:
    pickle.dump(model, file)

print("✅ Model saved successfully!")


✅ Model saved successfully!
